📝 Goal:
Take the name of a famous poet (e.g., Kafka, Dostoevsky, J.R. Rogue), and return:

🖋️ A summary of their writing style

🔍 Common themes and tone

🧱 Sentence structure & grammar style

🧠 Literary influence and uniqueness

In [4]:
# poet_analysis_agent.py

from langchain.tools.tavily_search import TavilySearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv

# ✅ Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# ✅ Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY
)

# ✅ Initialize Tavily Search
search_tool = TavilySearchResults(api_key=TAVILY_API_KEY)

# ✅ Prompt Template for Poet Analysis
poet_analysis_prompt = PromptTemplate(
    input_variables=["name", "context"],
    template="""
You are a literary analysis AI.

Given the following search context, analyze the writing style of the poet "{name}". 
Focus on:
- Common themes and emotional tones
- Unique language or metaphors
- Sentence structure or grammar quirks
- Influence and historical context (if relevant)

Present your analysis in a structured, paragraph format.

Context:
{context}

Summary:
"""
)

# ✅ Agent Function
def analyze_poet_style(poet_name: str) -> str:
    try:
        print(f"🔍 Searching for analysis on: {poet_name}")
        results = search_tool.run(f"{poet_name} poet writing style analysis")
        snippets = "\n".join([r['content'] for r in results[:5]])

        prompt = poet_analysis_prompt.format(name=poet_name, context=snippets)
        analysis = llm.invoke(prompt)
        
        return analysis

    except Exception as e:
        return f"❌ ERROR during poet analysis: {str(e)}"

# ✅ Example Usage
if __name__ == "__main__":
    poet = "JR Rougue"
    result = analyze_poet_style(poet)
    
    print(f"\n🧠 Literary Analysis for {poet}:\n")
    print(result.content)


🔍 Searching for analysis on: JR Rougue

🧠 Literary Analysis for JR Rougue:

J.R. Rogue's writing style is characterized by a delicate yet emotionally impactful approach to complex themes, particularly heartache and healing. The writing is described as "beautiful and poetic," with a serene pace and a "poetic cadence" that draws the reader in. Common themes revolve around raw emotions, trauma, and the arduous journey toward healing, often explored within the context of romantic relationships. The emotional tone is described as both warming and heart-wrenching, capable of "punch[ing] you right in the gut while ripping your heart wide open," suggesting a powerful and visceral reading experience.

Rogue’s language is evocative and metaphorical, as seen in the comparison to "orange and dry leaves carpets on the ground" in the context of falling in love with her writing. The narrative style avoids rushing, allowing events to unfold naturally and creating a captivating plot that balances the d

In [6]:
# poet_analysis_agent_v2.py

import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.tools.tavily_search import TavilySearchResults

load_dotenv()

class PoetAnalysisAgent:
    def __init__(self):
        self.google_api_key = os.getenv("GOOGLE_API_KEY")
        self.tavily_api_key = os.getenv("TAVILY_API_KEY")

        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash",
            google_api_key=self.google_api_key
        )

        self.search_tool = TavilySearchResults(api_key=self.tavily_api_key)

        self.summary_prompt = PromptTemplate(
            input_variables=["name", "context"],
            template="""
You are a literary analysis AI.

Based on the following context, write a detailed paragraph summarizing the poetic style and themes of "{name}". 
Include information on tone, grammar, themes, language, and historical influence.

Context:
{context}

Summary:
"""
        )

        self.refinement_prompt = PromptTemplate(
            input_variables=["summary"],
            template="""
You are a summary refinement expert.

Improve the following summary to make it more structured, poetic, and insightful. Make sure it's fluent, literary in tone, and free from errors.

Original Summary:
{summary}

Refined Summary:
"""
        )

    def search_poet_context(self, poet_name: str, max_results: int = 5) -> str:
        print(f"🔍 Searching for: {poet_name}")
        results = self.search_tool.run(f"{poet_name} poet writing style analysis")
        return "\n".join([r['content'] for r in results[:max_results]])

    def generate_summary(self, poet_name: str, context: str) -> str:
        prompt = self.summary_prompt.format(name=poet_name, context=context)
        return self.llm.invoke(prompt)

    def refine_summary(self, raw_summary: str) -> str:
        prompt = self.refinement_prompt.format(summary=raw_summary)
        return self.llm.invoke(prompt)

# ✅ Example Usage
if __name__ == "__main__":
    agent = PoetAnalysisAgent()
    poet_name = "Sylvia Plath"

    context = agent.search_poet_context(poet_name)
    raw_summary = agent.generate_summary(poet_name, context)
    refined_summary = agent.refine_summary(raw_summary)

    print("\n🧾 Final Refined Summary:\n")
    print(refined_summary.content)


🔍 Searching for: Sylvia Plath

🧾 Final Refined Summary:

Sylvia Plath, a luminary of confessional poetry, forged her legacy in verses of unvarnished emotion and stark honesty. Her pen plumbed the shadowed depths of the human psyche, grappling with the specters of mental illness, mortality, and the suffocating weight of despair. Themes of isolation and the fierce yearning for female autonomy echo throughout her oeuvre.

Plath's poetic voice, initially adorned with formal grace, evolved into a direct and emphatic cadence, often adopting a conversational intimacy that dissolved the distance between poet and reader. This deliberate simplification amplified the raw power of her later work.

Intensely autobiographical, her poems are hewn from the bedrock of her own life – the tempestuous currents of her marriage, the complex tapestry of familial relationships, and her singular self-perception. It is in this courageous act of self-revelation, stripping bare the polite facades and confronting 